Python libraries:
    rcsbsearchapi >>> functions for searching the Protein Data Bank based on the mmCIF dictionary,
    requests >>> access web URLs - used with APIs for databases,
    os >>> operating system functions - handling file paths and directories,
    nglview (nv) >>> for viewing molecular structures,
    MDAnalysis (mda) >>> molecular dynamics library - used for reading/writing files and selecting atoms

Softwares:
    pdb2pqr >>> adding hydrogens and missing atoms to protein, adjusting for pH
    meeko >>> preparing ligands for docking

Finding and saving a protein structure

In [3]:
from rcsbsearchapi.search import TextQuery
from rcsbsearchapi import rcsb_attributes as attrs

In [4]:
ECnumber = "3.4.21.4"   

q1 = attrs.rcsb_polymer_entity.rcsb_ec_lineage.id == ECnumber    # looking for trypsins
q2 = TextQuery("13U")     # looking for ligand

query = q1 & q2        # combining the two queries into one

results = list(query())
print(results)

['2ZQ2']


In [6]:
PDBID = results[0]
print(PDBID)

2ZQ2


In [7]:
pdbid = results[0].lower()      # Get the PDB ID from the list and convert it to lowercase
print(pdbid)

2zq2


In [8]:
import os    # for making directories
import requests

# make a directory for pdb files
protein_directory = "protein_structures"

# fill in function to make directories
os.makedirs(protein_directory, exist_ok = True)

In [9]:
pdb_request = requests.get(f"https://files.rcsb.org/download/{pdbid}.pdb")
pdb_request.status_code

200

In [10]:
with open(f'{protein_directory}/{pdbid}.pdb', "w+") as f:
    f.write(pdb_request.text)

# Visualizing the protein strucure

In [11]:
import MDAnalysis as mda

# Load into MDA universe
u = mda.Universe(f"{protein_directory}/{pdbid}.pdb")      # u for universe
u

/opt/conda/lib/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


<Universe with 2024 atoms>

In [12]:
import nglview as nv

# create and show an NGLView from an MDAnalysis universe
view = nv.show_mdanalysis(u)
view

NGLWidget()

In [18]:
#We will create separate variables for the protein, ligand and water.

# Select protein atoms
protein = u.select_atoms("protein")

# Select ligand atoms
ligand = u.select_atoms("resname 13U")

# Select water molecules
water = u.select_atoms("resname HOH")

In [19]:
protein

<AtomGroup with 1670 atoms>

In [20]:
ligand

<AtomGroup with 60 atoms>

In [21]:
water

<AtomGroup with 269 atoms>

In [22]:
1670 + 60 + 269

1999

In [23]:
prot_view = nv.show_mdanalysis(protein)
prot_view.clear_representations()
# Add representation to view protein surface colored by hydrophobicity
prot_view.add_representation("surface", colorScheme = "hydrophobicity")

lig_view = prot_view.add_component(ligand)
lig_view.add_representation("ball+stick")

water_view = prot_view.add_component(water)
water_view.add_representation("spacefill")

prot_view

NGLWidget()

In [24]:
# Write protein to new PDB file
protein.write(f"{protein_directory}/protein_{pdbid}.pdb")

/opt/conda/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1151: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


green areas of protein surface = hydrophilic 
red areas of protein surface = hydrophobic

Protein Charge: 
After saving the protein in the cell above, you may see a warning about information for formal charges not being set in the protein. This warning appears because MDAnalysis did not find specific formal charge data in the PDB file and used a default value instead. This is not a concern for us because we will adjust the protonation states of different residues using PDB2PQR in the next steps.

# Fixing the protein structure

We will want to ensure that we've correctly added hydrogen and fixed any missing atoms.

We will use the command-line interface of PDB2PQR. 
This means that you would usually type the command below into your terminal. 
You can run command line commands in the Jupyter notebook 
by putting a ! in front of the command.

In [25]:
! pdb2pqr --pdb-output=protein_structures/protein_h.pdb --pH=7.4 protein_structures/protein_2zq2.pdb protein_structures/protein_2zq2.pqr --whitespace

INFO:PDB2PQR v3.6.1: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: protein_structures/protein_2zq2.pdb
ERROR:Error parsing line: invalid literal for int() with base 10: ''
ERROR:<REMARK     2>
ERROR:Truncating remaining errors for record type:REMARK

ERROR:['REMARK']
INFO:Setting up molecule.
INFO:Created biomolecule object with 223 residues and 1625 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:Attempting to repair 4 missing atoms in biomolecule.
INFO:Added atom CG to residue LYS A 222 at

# Saving a protein PDBQT File

The PDB2PQR program outputs two files, a PDB file and a PQR file. The PDB file is similar to PDB files we have worked with before, except that it contains hydrogens. The PQR file is another molecular file format that is similar to a PDB, but contains information about atomic radii and atomic charges.

For use with AutoDock Vina, we need our protein file to be in the "PDBQT" format. PDBQT is a specialized file format used by AutoDock Vina and other AutoDock tools. Like the PQR format, the PDBQT format can also contain partial charges. We will load our PQR file and use MDAnalysis to write a PDBQT file.

In [26]:
# make a directory for pdbqt files
pdbqt_directory = "pdbqt"
os.makedirs(pdbqt_directory, exist_ok = True)

In [27]:
u = mda.Universe(f"{protein_directory}/protein_{pdbid}.pqr")
u.atoms.write(f"{pdbqt_directory}/{pdbid}.pdbqt")

/opt/conda/lib/python3.11/site-packages/MDAnalysis/coordinates/PDBQT.py:295: UserWarning: Supplied AtomGroup was missing the following attributes: altLocs, occupancies, tempfactors. These will be written with default values. 
  warnings.warn(


The PDBQT file generated by MDAnalysis includes two lines at the start of the structure that AutoDock Vina doesn't accept. These lines start with "TITLE" and "CRYST1". To resolve this, the following cell replaces these lines with "REMARK", which is acceptable to AutoDock Vina.

In [28]:
# Read in the just-written PDBQT file
with open(f'{pdbqt_directory}/{pdbid}.pdbqt') as file:
    file_content = file.read()

In [29]:
# Replace 'TITLE' and 'CRYST1' with 'REMARK'
mod_file_content = file_content.replace('TITLE', 'REMARK').replace('CRYST1', 'REMARK')

In [30]:
# Write the modified content back to the file
with open(f'{pdbqt_directory}/{pdbid}.pdbqt', 'w') as file:
    file.write(mod_file_content)

# Ligand Preparation

We are going to use a special program for small molecules and docking called meeko. We choose to use meeko for our ligands because it will allow us to more easily visualize our results later. Note that when using meeko, ligands should have hydrogens added already.

We are using the command line for meeko to convert an SDF to a PDBQT.

In the cell below, we execute a command that converts our sdf ligands that we prepared in the molecule_manipulation notebook to a PDBQT file.

In [33]:
# Use meeko to prepare small molecules 
# using meeko helps us visualize them later
# ! mk_prepare_ligand.py -i input -o output

! mk_prepare_ligand.py -i ligands_to_dock/13U.sdf -o pdbqt/13U.pdbqt
! mk_prepare_ligand.py -i ligands_to_dock/13U_modified_methyl.sdf -o pdbqt/13U_modified_methyl.pdbqt
! mk_prepare_ligand.py -i ligands_to_dock/13U_modified_N.sdf -o pdbqt/13U_modified_N.pdbqt

Input molecules processed: 1, skipped: 0
PDBQT files written: 1
PDBQT files not written due to error: 0
Input molecules with errors: 0
Input molecules processed: 1, skipped: 0
PDBQT files written: 1
PDBQT files not written due to error: 0
Input molecules with errors: 0
Input molecules processed: 1, skipped: 0
PDBQT files written: 1
PDBQT files not written due to error: 0
Input molecules with errors: 0
